In [1]:
# # Geografia & Inequidades – Gráficos interativos
# 
# Este script:
# - Lê df_tipado.csv
# - Calcula resumos por UF, região, capital/interior, zona e raça
# - Gera gráficos interativos com Plotly

In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
from pathlib import Path
pd.set_option("display.max_columns", 120)

# Caminhos Utilizados
PROJECT_ROOT = Path.cwd().parent 
DATA_PROCESSED = PROJECT_ROOT / "srag_2023_analysis" / "data" / "processed" 
CSV_PATH = DATA_PROCESSED / "df_tipado.csv"

In [6]:
df = pd.read_csv(CSV_PATH,low_memory=False)
print("Shape df_tipado:", df.shape)
df.head()

Shape df_tipado: (279453, 176)


,NU_NOTIFIC,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,CS_SEXO,DT_NASC,NU_IDADE_N,TP_IDADE,COD_IDADE,CS_GESTANT,CS_RACA,CS_ESCOL_N,ID_PAIS,CO_PAIS,SG_UF,ID_RG_RESI,CO_RG_RESI,ID_MN_RESI,CO_MUN_RES,CS_ZONA,NOSOCOMIAL,AVE_SUINO,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,OUTRO_DES,FATOR_RISC,PUERPERA,CARDIOPATI,HEMATOLOGI,SIND_DOWN,HEPATICA,ASMA,DIABETES,NEUROLOGIC,PNEUMOPATI,IMUNODEPRE,RENAL,OBESIDADE,OUT_MORBI,MORB_DESC,VACINA,MAE_VAC,M_AMAMENTA,ANTIVIRAL,TP_ANTIVIR,DT_ANTIVIR,HOSPITAL,...,FAB_COV_2,FAB_COVRF,FAB_COVRF2,FAB_RE_BI,LOTE_1_COV,LOTE_2_COV,LOTE_REF,LOTE_REF2,LOT_RE_BI,FNT_IN_COV,TRAT_COV,SURTO_SG,CO_DETEC,REINF,CS_SEXO_label,CS_GESTANT_label,CS_RACA_label,CS_ESCOL_N_label,CS_ZONA_label,EVOLUCAO_label,UTI_label,SUPORT_VEN_label,CS_SEXO_clean,CS_RACA_macro,NU_IDADE_N_num,CS_ESCOL_N_label_qualidade,EVOLUCAO_BIN,EVOLUCAO_disponivel,UTI_flag,UTI_missing,UTI_ignorado,GRAVIDADE_RESPI,DT_NOTIFIC_date,DT_SIN_PRI_date,DT_NASC_date,DT_ANTIVIR_date,DT_INTERNA_date,DT_ENTUTI_date,DT_SAIDUTI_date,DT_RAIOX_date,DT_COLETA_date,DT_PCR_date,DT_EVOLUCA_date,DT_ENCERRA_date,DT_DIGITA_date,DT_TOMO_date,DT_RES_AN_date,PUERPERA_flag,CARDIOPATI_flag,HEMATOLOGI_flag,SIND_DOWN_flag,HEPATICA_flag,ASMA_flag,DIABETES_flag,NEUROLOGIC_flag,PNEUMOPATI_flag,IMUNODEPRE_flag,RENAL_flag,OBESIDADE_flag,OUT_MORBI_flag
0,316004370301,2023-02-07,6,2023-02-05,6,SP,GVE VII SANTO ANDRE,1332.0,SAO CAETANO DO SUL,354880.0,F,1941-06-30,81,3,3081,6,1,1.0,BRASIL,1,SP,GVE VII SANTO ANDRE,1332.0,SAO CAETANO DO SUL,354880.0,1.0,2.0,2.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,RNC,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1.0,ACAMADA HA 12 ANOS,2.0,NaN,NaN,2.0,NaN,NaN,1.0,...,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,88 - COVID-19 JANSSEN - AD26.COV2.S,88 - COVID-19 JANSSEN - AD26.COV2.S,NaN,202010034,210083,210F21A,210F21A,NaN,2,NaN,NaN,NaN,2.0,F - Feminino,6 - Não se aplica,1 - Branca,1 - Fundamental 1º ciclo (1ª a,1 - Urbana,2 - Óbito,2 - Não,3 - Não,F,Branca,81,1 - Fundamental 1º ciclo (1ª a,1.0,1,0.0,0,0,0.0,2023-02-07,2023-02-05,1941-06-30,NaN,2023-02-07,NaN,NaN,NaN,2023-02-07,NaN,2023-02-08,2023-02-09,2020-09-18,NaN,2023-02-07,NaN,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1.0
1,316153805701,2023-02-28,9,2023-01-17,3,SP,GVE VII SANTO ANDRE,1332.0,SAO CAETANO DO SUL,354880.0,M,1933-12-10,89,3,3089,6,1,1.0,BRASIL,1,SP,GVE VII SANTO ANDRE,1332.0,SAO CAETANO DO SUL,354880.0,1.0,2.0,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,QUEDA DO ESTADO GERAL,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NEO PROSTATA/HIPOTIREOIDISMO,2.0,NaN,NaN,2.0,NaN,NaN,1.0,...,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,NaN,NaN,202010034,210062,210448,NaN,NaN,2,NaN,NaN,NaN,2.0,M - Masculino,6 - Não se aplica,1 - Branca,1 - Fundamental 1º ciclo (1ª a,1 - Urbana,2 - Óbito,2 - Não,3 - Não,M,Branca,89,1 - Fundamental 1º ciclo (1ª a,1.0,1,0.0,0,0,0.0,2023-02-28,2023-01-17,1933-12-10,NaN,2023-01-17,NaN,NaN,NaN,2023-01-17,NaN,2023-02-26,2023-02-28,2021-03-10,NaN,2023-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,316321420027,2023-05-02,18,2023-04-14,15,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690.0,F,1943-06-04,78,3,3078,5,1,9.0,BRASIL,1,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690.0,1.0,2.0,2.0,NaN,1.0,1.0,1.0,NaN,2.0,NaN,NaN,1.0,"CEFALEIA, ASTENIA",1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,HIPERTENSAO ARTERIAL,2.0,NaN,NaN,2.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,210056,210129,NaN,NaN,NaN,1,NaN,2.0,2.0,2.0,F - Feminino,5 - Não,1 - Branca,9 - Ignorado,1 - Urbana,1 - Cura,2 - Não,"2 - Sim, não invasivo",F,Branca,78,9 - Ignorado,0.0,1,0.0,0,0,1.0,2023-05-02,2023-04-14,1943-06-04,NaN,2023-05-12,NaN,NaN,2023-04-30,2023-05-02,2023-05-02,2023-05-15,2023-05-16,2021-09-20,2021-09-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,316384665515,2023-03-09,10,2023-03-05,10,SP,GVE VII SANTO ANDRE,1332.0,SAO CAETANO DO SUL,354880.0,F,1916-04-

In [7]:
# ==== 1. Configurações de colunas ====
UF_COL = "SG_UF_NOT"
MUN_COL = "ID_MN_RESI"          # ajuste se o nome for outro
ZONA_LABEL_COL = "CS_ZONA_label"
RACA_MACRO_COL = "CS_RACA_macro"

In [8]:
# Padroniza
df[UF_COL] = df[UF_COL].astype(str).str.upper().str.strip()
df[MUN_COL] = df[MUN_COL].astype(str).str.upper().str.strip()
df["EVOLUCAO_BIN"] = pd.to_numeric(df["EVOLUCAO_BIN"], errors="coerce")
df["EVOLUCAO_disponivel"] = pd.to_numeric(df["EVOLUCAO_disponivel"], errors="coerce")

In [9]:
# Região a partir da UF
REGIAO_MAP = {
    "AC": "N", "AP": "N", "AM": "N", "PA": "N", "RO": "N", "RR": "N", "TO": "N",
    "AL": "NE", "BA": "NE", "CE": "NE", "MA": "NE", "PB": "NE", "PE": "NE",
    "PI": "NE", "RN": "NE", "SE": "NE",
    "ES": "SE", "MG": "SE", "RJ": "SE", "SP": "SE",
    "PR": "S", "RS": "S", "SC": "S",
    "DF": "CO", "GO": "CO", "MS": "CO", "MT": "CO",
}
df["REGIAO"] = df[UF_COL].map(REGIAO_MAP)


In [10]:
# Capital x interior
CAPITAIS_POR_UF = {
    "AC": "RIO BRANCO",
    "AL": "MACEIO",
    "AM": "MANAUS",
    "AP": "MACAPA",
    "BA": "SALVADOR",
    "CE": "FORTALEZA",
    "DF": "BRASILIA",
    "ES": "VITORIA",
    "GO": "GOIANIA",
    "MA": "SAO LUIS",
    "MG": "BELO HORIZONTE",
    "MS": "CAMPO GRANDE",
    "MT": "CUIABA",
    "PA": "BELEM",
    "PB": "JOAO PESSOA",
    "PE": "RECIFE",
    "PI": "TERESINA",
    "PR": "CURITIBA",
    "RJ": "RIO DE JANEIRO",
    "RN": "NATAL",
    "RO": "PORTO VELHO",
    "RR": "BOA VISTA",
    "RS": "PORTO ALEGRE",
    "SC": "FLORIANOPOLIS",
    "SE": "ARACAJU",
    "SP": "SAO PAULO",
    "TO": "PALMAS",
}

df["TIPO_MUN"] = np.where(
    (df[UF_COL].isin(CAPITAIS_POR_UF.keys())) &
    (df[MUN_COL] == df[UF_COL].map(CAPITAIS_POR_UF)),
    "capital",
    "interior"
)


In [11]:
# Zona e raça
if ZONA_LABEL_COL in df.columns:
    df[ZONA_LABEL_COL] = df[ZONA_LABEL_COL].fillna("Missing")

if RACA_MACRO_COL in df.columns:
    df[RACA_MACRO_COL] = df[RACA_MACRO_COL].fillna("Ignorado/Missing")


In [12]:
# ==== 2. Função de resumo geográfico ====

def resumo_geo(df_in, group_cols, uf_col=UF_COL):
    """
    Retorna:
    - n_casos
    - n_obitos
    - letalidade (0-1) e letalidade_pct (0-100)
    - pct_casos_total (% dos casos da amostra)
    """
    d = df_in.copy()
    d = d[~d[uf_col].isna()].copy()

    d["caso_flag"] = 1
    d["obito_flag"] = np.where(
        (d["EVOLUCAO_disponivel"] == 1) & (d["EVOLUCAO_BIN"] == 1),
        1, 0
    )

    grp = (
        d.groupby(group_cols, dropna=False)
         .agg(
             n_casos=("caso_flag", "sum"),
             n_obitos=("obito_flag", "sum"),
         )
         .reset_index()
    )

    grp["letalidade"] = grp["n_obitos"] / grp["n_casos"]
    grp["letalidade_pct"] = grp["letalidade"] * 100
    total_casos = grp["n_casos"].sum()
    grp["pct_casos_total"] = 100 * grp["n_casos"] / total_casos

    return grp


# ==== 3. Resumos ====

resumo_uf = resumo_geo(df, [UF_COL])
resumo_uf = resumo_uf.merge(
    df[[UF_COL, "REGIAO"]].drop_duplicates(),
    on=UF_COL,
    how="left"
)

resumo_regiao = resumo_geo(df, ["REGIAO"])
resumo_uf_tipo = resumo_geo(df, [UF_COL, "TIPO_MUN"])

resumo_zona = resumo_geo(df, [ZONA_LABEL_COL]) if ZONA_LABEL_COL in df.columns else None
resumo_zona_regiao = (
    resumo_geo(df, ["REGIAO", ZONA_LABEL_COL]) if ZONA_LABEL_COL in df.columns else None
)

resumo_raca_regiao = (
    resumo_geo(df, ["REGIAO", RACA_MACRO_COL]) if RACA_MACRO_COL in df.columns else None
)
tabela_raca_regiao = None
if resumo_raca_regiao is not None:
    tabela_raca_regiao = (
        resumo_raca_regiao
        .pivot(index=RACA_MACRO_COL, columns="REGIAO", values="letalidade_pct")
        .sort_index()
    )

resumo_raca_tipo = (
    resumo_geo(df, ["TIPO_MUN", RACA_MACRO_COL]) if RACA_MACRO_COL in df.columns else None
)

In [13]:
# ==== 4. Gráficos interativos ====

# 4.1 Casos x letalidade por UF (colorindo por região)
fig_uf_scatter = px.scatter(
    resumo_uf,
    x="n_casos",
    y="letalidade_pct",
    color="REGIAO",
    size="n_obitos",
    hover_name=UF_COL,
    hover_data={"n_casos": True, "n_obitos": True, "letalidade_pct": ":.2f"},
    labels={
        "n_casos": "Número de casos",
        "letalidade_pct": "Letalidade (%)",
        "REGIAO": "Região"
    },
    title="Casos x letalidade de SRAG por UF (cor = região, tamanho = óbitos)"
)
fig_uf_scatter.show()


# 4.2 Letalidade por região (barra)
fig_regiao_bar = px.bar(
    resumo_regiao.sort_values("letalidade_pct", ascending=False),
    x="REGIAO",
    y="letalidade_pct",
    hover_data={"n_casos": True, "n_obitos": True, "pct_casos_total": ":.1f"},
    labels={
        "REGIAO": "Região",
        "letalidade_pct": "Letalidade (%)"
    },
    title="Letalidade (%) e carga de casos por região"
)
fig_regiao_bar.update_traces(texttemplate="%{y:.1f}%", textposition="outside")
fig_regiao_bar.update_layout(uniformtext_minsize=8, uniformtext_mode="hide")
fig_regiao_bar.show()


# 4.3 Letalidade por UF x tipo de município (capital x interior)
fig_uf_tipo = px.bar(
    resumo_uf_tipo,
    x=UF_COL,
    y="letalidade_pct",
    color="TIPO_MUN",
    barmode="group",
    hover_data={"n_casos": True, "n_obitos": True, "letalidade_pct": ":.2f"},
    labels={
        UF_COL: "UF",
        "letalidade_pct": "Letalidade (%)",
        "TIPO_MUN": "Tipo de município"
    },
    title="Letalidade de SRAG por UF e tipo de município (capital x interior)"
)
fig_uf_tipo.show()


# 4.4 Zona (urbana/rural/periurbana) – distribuição de casos e letalidade
if resumo_zona is not None:
    fig_zona = px.bar(
        resumo_zona.sort_values("pct_casos_total", ascending=False),
        x=ZONA_LABEL_COL,
        y="pct_casos_total",
        hover_data={"n_casos": True, "n_obitos": True, "letalidade_pct": ":.2f"},
        labels={
            ZONA_LABEL_COL: "Zona",
            "pct_casos_total": "% dos casos"
        },
        title="Distribuição de casos (%) e letalidade por zona"
    )
    fig_zona.update_traces(texttemplate="%{y:.1f}%", textposition="outside")
    fig_zona.update_layout(uniformtext_minsize=8, uniformtext_mode="hide")
    fig_zona.show()


# 4.5 Heatmap de letalidade por macro-raça x região
if tabela_raca_regiao is not None:
    fig_raca_regiao = px.imshow(
        tabela_raca_regiao,
        text_auto=".1f",
        aspect="auto",
        color_continuous_scale="RdBu_r",
        origin="lower",
        labels={"color": "Letalidade (%)"},
        title="Letalidade (%) por macro-raça x região"
    )
    fig_raca_regiao.show()


# 4.6 Letalidade por macro-raça x tipo de município (capital x interior)
if resumo_raca_tipo is not None:
    fig_raca_tipo = px.bar(
        resumo_raca_tipo,
        x=RACA_MACRO_COL,
        y="letalidade_pct",
        color="TIPO_MUN",
        barmode="group",
        hover_data={"n_casos": True, "n_obitos": True, "letalidade_pct": ":.2f"},
        labels={
            RACA_MACRO_COL: "Macro-raça",
            "letalidade_pct": "Letalidade (%)",
            "TIPO_MUN": "Tipo de município"
        },
        title="Letalidade de SRAG por macro-raça e tipo de município"
    )
    fig_raca_tipo.show()